In [39]:
import csv
import pandas as pd
import datetime
import calendar
import folium

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CQ/Crime_Data_from_2020_to_Present.csv')

In [42]:
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])
df['DAY_OF_WEEK'] = df['DATE OCC'].dt.day_name()
print(df['DAY_OF_WEEK'])

<ipython-input-42-daa7d35b1971>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])


0             Sunday
1           Saturday
2          Wednesday
3            Tuesday
4          Wednesday
             ...    
1004842      Tuesday
1004843       Monday
1004844       Friday
1004845    Wednesday
1004846       Monday
Name: DAY_OF_WEEK, Length: 1004847, dtype: object


In [43]:
# Clean and standardize the 'Crm Cd Desc' column
df['Crm Cd Desc'] = df['Crm Cd Desc'].str.strip()  # Remove leading/trailing whitespace
df['Crm Cd Desc'] = df['Crm Cd Desc'].str.upper()  # Convert to uppercase for case insensitivity

In [44]:
# Group by AREA NAME and Crime Type, then count occurrences
crime_counts = df.groupby(['AREA NAME', 'Crm Cd Desc']).size().reset_index(name='Count')

# Sort by AREA NAME and Count (descending) to get most common first
crime_counts = crime_counts.sort_values(['AREA NAME', 'Count'], ascending=[True, False])

# Get the most common crime in each area (first row per group)
most_common_crimes = crime_counts.loc[crime_counts.groupby('AREA NAME')['Count'].idxmax()]

# Reset index for cleaner output
most_common_crimes = most_common_crimes.reset_index(drop=True)

# Display result
print(most_common_crimes)
print(crime_counts)

      AREA NAME               Crm Cd Desc  Count
0   77th Street          VEHICLE - STOLEN   8766
1       Central     BURGLARY FROM VEHICLE   9695
2    Devonshire          VEHICLE - STOLEN   3959
3      Foothill          VEHICLE - STOLEN   4539
4        Harbor          VEHICLE - STOLEN   6137
5    Hollenbeck          VEHICLE - STOLEN   6213
6     Hollywood  BATTERY - SIMPLE ASSAULT   4482
7       Mission          VEHICLE - STOLEN   6050
8   N Hollywood          VEHICLE - STOLEN   5106
9        Newton          VEHICLE - STOLEN   8281
10    Northeast          VEHICLE - STOLEN   5348
11      Olympic          VEHICLE - STOLEN   5731
12      Pacific          VEHICLE - STOLEN   6521
13      Rampart          VEHICLE - STOLEN   5441
14    Southeast          VEHICLE - STOLEN   7242
15    Southwest          VEHICLE - STOLEN   6649
16      Topanga          VEHICLE - STOLEN   3457
17     Van Nuys          VEHICLE - STOLEN   4683
18      West LA                  BURGLARY   4378
19  West Valley     

In [45]:
# Apply severity mapping based on the crime description
most_common_crimes["Severity"] = most_common_crimes["Crm Cd Desc"].map(crime_to_severity)

# Fill missing severity values with 'Uncategorized' if no crime type is present
most_common_crimes["Severity"].fillna("Uncategorized", inplace=True)

# Dictionary of LAPD areas with coordinates
lapd_areas = {
    "Wilshire": (34.0561, -118.3406),
    "Central": (34.0505, -118.2478),
    "Southwest": (34.0182, -118.3082),
    "Van Nuys": (34.1850, -118.4489),
    "Hollenbeck": (34.0446, -118.2115),
    "Rampart": (34.0651, -118.2760),
    "Newton": (34.0039, -118.2580),
    "Northeast": (34.1070, -118.2110),
    "77th Street": (33.9702, -118.2733),
    "Hollywood": (34.1016, -118.3269),
    "Harbor": (33.7701, -118.2936),
    "West Valley": (34.1867, -118.5533),
    "West LA": (34.0453, -118.4426),
    "N Hollywood": (34.1870, -118.3753),
    "Pacific": (33.9606, -118.4194),
    "Devonshire": (34.2570, -118.5500),
    "Mission": (34.2724, -118.4390),
    "Southeast": (33.9442, -118.2468),
    "Olympic": (34.0560, -118.2910),
    "Foothill": (34.2768, -118.4136),
    "Topanga": (34.2103, -118.6056)
}

# Create base map centered in LA
map_la = folium.Map(location=[34.0522, -118.2437], zoom_start=11)

# Add markers with popups
for area, (lat, lon) in lapd_areas.items():
    # Get most common crime & severity for the area
    crime_info = most_common_crimes[most_common_crimes["AREA NAME"] == area]

    if not crime_info.empty:
        crime_type = crime_info.iloc[0]["Crm Cd Desc"]
        severity = crime_info.iloc[0]["Severity"]
    else:
        crime_type, severity = "No Data", "Uncategorized"

    # HTML Table for Popup
    popup_html = f"""
    <table border="1" style="width:250px">
        <tr><th>Area</th><td>{area}</td></tr>
        <tr><th>Most Common Crime</th><td>{crime_type}</td></tr>
        <tr><th>Severity</th><td>{severity}</td></tr>
    </table>
    """

    # Assign color based on severity
    if severity == "Severe":
        marker_color = "red"
    elif severity == "High":
        marker_color = "orange"
    elif severity == "Moderate":
        marker_color = "yellow"
    elif severity == "Low":
        marker_color = "blue"
    else:  # Uncategorized or Low severity
        marker_color = "green"

    # Add marker with popup
    folium.Marker(
        location=(lat, lon),
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=marker_color)
    ).add_to(map_la)

# Save and display the map
map_la.save("lapd_crime_map.html")
map_la


<ipython-input-45-dc4abcc48b29>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  most_common_crimes["Severity"].fillna("Uncategorized", inplace=True)
<ipython-input-45-dc4abcc48b29>:71: UserWarning: color argument of Icon should be one of: {'lightgray', 'darkpurple', 'cadetblue', 'darkred', 'orange', 'pink', 'white', 'black', 'red', 'green', 'darkblue', 'lightred', 'gray', 'blue', 'lightblue', 'darkgreen', 'lightgreen', 'purple', 'beige'}.
  icon=folium.Icon(color=marker_color)
